In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import os

import haversine as hs

from datetime import datetime, timedelta

import dateutil.parser

In [2]:
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
race_info = pd.read_csv("Race_Info.csv")

race_info.shape

(1082, 15)

In [4]:
race_info.head()

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2023,1,Bahrain Grand Prix,2023-03-05,Bahrain,Sakhir,Bahrain International Circuit,bahrain,single,0,0.00,26.0325,50.5106,1098,3
1,2023,2,Saudi Arabian Grand Prix,2023-03-19,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,single,2,1258.43,21.6319,39.1044,1099,77
2,2023,3,Australian Grand Prix,2023-04-02,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.9680,1100,1
3,2023,4,Azerbaijan Grand Prix,2023-04-30,Azerbaijan,Baku,Baku City Circuit,baku,double,4,12989.08,40.3725,49.8533,1101,73
4,2023,5,Miami Grand Prix,2023-05-07,USA,Miami,Miami International Autodrome,miami,double,1,11015.93,25.9581,-80.2389,1102,79


In [5]:
race_info.loc[race_info['year'] == 2022]

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
22,2022,1,Bahrain Grand Prix,2022-03-20,Bahrain,Sakhir,Bahrain International Circuit,bahrain,double,0,0.00,26.0325,50.51060,1074,3
23,2022,2,Saudi Arabian Grand Prix,2022-03-27,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,double,1,1258.43,21.6319,39.10440,1075,77
24,2022,3,Australian Grand Prix,2022-04-10,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1076,1
25,2022,4,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
26,2022,5,Miami Grand Prix,2022-05-08,USA,Miami,Miami International Autodrome,miami,single,2,8172.92,25.9581,-80.23890,1078,79
27,2022,6,Spanish Grand Prix,2022-05-22,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,2,7536.29,41.5700,2.26111,1079,4
28,2022,7,Monaco Grand Prix,2022-05-29,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,1,485.64,43.7347,7.42056,1080,6
29,2022,8,Azerbaijan Grand Prix,2022-06-12,Azerbaijan,Baku,Baku City Circuit,baku,double,2,3484.89,40.3725,49.85330,1081,73
30,2022,9,Canadian Grand Prix,2022-06-19,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,double,1,8930.53,45.5000,-73.52280,1082,7
31,2022,10,British Grand Prix,2022-07-03,UK,Silverstone,Silverstone Circuit,silverstone,double,2,5137.22,52.0786,-1.01694,1083,9


In [7]:
f1_22 = race_info.loc[race_info['year'] == 2022]

f1_22.reset_index(drop = True, inplace = True)

f1_22

,year,round,race,date,country,location,circuit,circuitRef,header,gap,distance (km),lat,lng,raceId,circuitId
0,2022,1,Bahrain Grand Prix,2022-03-20,Bahrain,Sakhir,Bahrain International Circuit,bahrain,double,0,0.00,26.0325,50.51060,1074,3
1,2022,2,Saudi Arabian Grand Prix,2022-03-27,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,double,1,1258.43,21.6319,39.10440,1075,77
2,2022,3,Australian Grand Prix,2022-04-10,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,single,2,12817.12,-37.8497,144.96800,1076,1
3,2022,4,Emilia Romagna Grand Prix,2022-04-24,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,single,2,16086.36,44.3439,11.71670,1077,21
4,2022,5,Miami Grand Prix,2022-05-08,USA,Miami,Miami International Autodrome,miami,single,2,8172.92,25.9581,-80.23890,1078,79
5,2022,6,Spanish Grand Prix,2022-05-22,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,double,2,7536.29,41.5700,2.26111,1079,4
6,2022,7,Monaco Grand Prix,2022-05-29,Monaco,Monte-Carlo,Circuit de Monaco,monaco,double,1,485.64,43.7347,7.42056,1080,6
7,2022,8,Azerbaijan Grand Prix,2022-06-12,Azerbaijan,Baku,Baku City Circuit,baku,double,2,3484.89,40.3725,49.85330,1081,73
8,2022,9,Canadian Grand Prix,2022-06-19,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,double,1,8930.53,45.5000,-73.52280,1082,7
9,2022,10,British Grand Prix,2022-07-03,UK,Silverstone,Silverstone Circuit,silverstone,double,2,5137.22,52.0786,-1.01694,1083,9


In [9]:
tracks = f1_22['circuitRef'].values.tolist()

print(tracks)

['bahrain', 'jeddah', 'albert_park', 'imola', 'miami', 'catalunya', 'monaco', 'baku', 'villeneuve', 'silverstone', 'red_bull_ring', 'ricard', 'hungaroring', 'spa', 'zandvoort', 'monza', 'marina_bay', 'suzuka', 'americas', 'rodriguez', 'interlagos', 'yas_marina']


In [10]:
ext = '.csv'

for track in tracks:
    print(track + ext)

bahrain.csv
jeddah.csv
albert_park.csv
imola.csv
miami.csv
catalunya.csv
monaco.csv
baku.csv
villeneuve.csv
silverstone.csv
red_bull_ring.csv
ricard.csv
hungaroring.csv
spa.csv
zandvoort.csv
monza.csv
marina_bay.csv
suzuka.csv
americas.csv
rodriguez.csv
interlagos.csv
yas_marina.csv


In [11]:
datapath = "Weather/"

In [12]:
os.listdir(datapath)

['zandvoort.csv',
 'americas.csv',
 'yas_marina.csv',
 'hungaroring.csv',
 'losail.csv',
 'miami.csv',
 'marina_bay.csv',
 'interlagos.csv',
 'spa.csv',
 'bahrain.csv',
 'jeddah.csv',
 'baku.csv',
 'vegas.csv',
 'red_bull_ring.csv',
 'imola.csv',
 'monaco.csv',
 'villeneuve.csv',
 'catalunya.csv',
 'albert_park.csv',
 'suzuka.csv',
 'shanghai.csv',
 'silverstone.csv',
 'monza.csv',
 'rodriguez.csv']

In [44]:
f = '%Y/%m/%d'

f1_22['date'] = pd.to_datetime(f1_22['date'], format=f).dt.date

In [53]:
track = 'yas_marina'

track_weather = pd.read_csv(datapath + track + ext)

race_date = f1_22.loc[f1_22['circuitRef'] == track, 'date'].values[0]

print(datapath + track + ext, race_date)

#track_weather['time'].tail(50)

f = '%d/%m/%Y'

track_weather['date'] = track_weather['time']

track_weather['date'] = pd.to_datetime(track_weather['date'], format=f).dt.date

race_weather = track_weather.loc[track_weather['date'] == race_date]
     
print() 

print(race_date, race_weather)   

#track_weather.tail(50)

print(type(race_date), race_date)

#track_weather.info()


Weather/yas_marina.csv 2022-11-20

2022-11-20             time  weathercode (wmo code)  temperature_2m_max (°C)  \
3610  20/11/2022                       0                     31.3   

      temperature_2m_min (°C)  temperature_2m_mean (°C)  \
3610                     23.7                      27.3   

      apparent_temperature_max (°C)  apparent_temperature_min (°C)  \
3610                           30.7                           25.9   

      apparent_temperature_mean (°C) sunrise (iso8601)  sunset (iso8601)  \
3610                            28.0  2022-11-20T06:40  2022-11-20T17:33   

      precipitation_sum (mm)  rain_sum (mm)  snowfall_sum (cm)  \
3610                     0.0            0.0                  0   

      precipitation_hours (h)  windspeed_10m_max (km/h)  \
3610                        0                      28.9   

      windgusts_10m_max (km/h)  winddirection_10m_dominant (°)  \
3610                      43.2                             131   

      shortwave_r

In [54]:
race_weather

,time,weathercode (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),temperature_2m_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),apparent_temperature_mean (°C),sunrise (iso8601),sunset (iso8601),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),precipitation_hours (h),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),shortwave_radiation_sum (MJ/m²),et0_fao_evapotranspiration (mm),date
3610,20/11/2022,0,31.3,23.7,27.3,30.7,25.9,28.0,2022-11-20T06:40,2022-11-20T17:33,0.0,0.0,0,0,28.9,43.2,131,16.42,5.28,2022-11-20


In [55]:
f = '%Y/%m/%d'

f1_22['date'] = pd.to_datetime(f1_22['date'], format=f).dt.date

In [64]:
f1_22_weather = pd.DataFrame()
f = '%d/%m/%Y'

for track in tracks:
    
    race_date = f1_22.loc[f1_22['circuitRef'] == track, 'date'].values[0]
    
    try:
        track_weather = pd.read_csv(datapath + track + ext)
        
    except:
        print("No file :", datapath + track + ext)
        
        continue
        
    track_weather['date'] = track_weather['time']

    track_weather['date'] = pd.to_datetime(track_weather['date'], format=f).dt.date
    
    race_weather = track_weather.loc[track_weather['date'] == race_date]
     
    #print(race_date, race_weather['date'])    
        
    f1_22_weather = pd.concat([f1_22_weather, race_weather])
    
    f1_22_weather.loc[f1_22_weather['date'] == race_date, 'circuitRef'] = track
    
    
    
    #print(datapath + track + ext, race_date)
    
    #print()

No file : Weather/ricard.csv


In [65]:
f1_22_weather

,time,weathercode (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),temperature_2m_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),apparent_temperature_mean (°C),sunrise (iso8601),sunset (iso8601),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),precipitation_hours (h),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),shortwave_radiation_sum (MJ/m²),et0_fao_evapotranspiration (mm),date,circuitRef
3365,20/03/2022,0,22.8,17.3,20.000000,22.2,14.6,17.900000,2022-03-20T05:41,2022-03-20T17:48,0.0,0.0,0.0,0,22.5,29.2,127.00000,24.07,5.510000,2022-03-20,bahrain
3372,27/03/2022,0,30.4,19.6,24.700000,30.0,19.5,23.600000,2022-03-27T06:21,2022-03-27T18:36,0.0,0.0,0.0,0,28.3,39.2,121.00000,25.53,7.000000,2022-03-27,jeddah
3386,10/04/2022,2,26.4,17.0,21.100000,24.3,15.9,19.700000,2022-04-10T06:42,2022-04-10T18:01,0.0,0.0,0.0,0,25.8,47.2,94.00000,15.47,3.980000,2022-04-10,albert_park
3400,24/04/2022,61,16.5,10.7,13.700000,14.5,8.8,11.900000,2022-04-24T06:14,2022-04-24T20:08,3.3,3.3,0.0,9,18.0,46.4,246.00000,12.52,2.260000,2022-04-24,imola
3414,08/05/2022,53,30.3,23.3,26.000000,35.7,26.0,29.800000,2022-05-08T06:38,2022-05-08T19:56,2.7,2.7,0.0,11,16.6,34.2,201.00000,21.23,4.390000,2022-05-08,miami
3428,22/05/2022,1,30.8,17.1,24.320833,29.8,16.0,23.254166,2022-05-22T06:25,2022-05-22T21:09,0.0,0.0,0.0,0,21.4,35.6,212.98096,29.01,7.179999,2022-05-22,catalunya
3435,29/05/2022,53,22.1,18.8,20.800000,24.3,20.0,22.200000,2022-05-29T05:53,2022-05-29T21:02,5.3,5.3,0.0,14,10.3,34.6,321.00000,19.20,3.430000,2022-05-29,monaco
3449,12/06/2022,1,27.9,21.4,25.000000,28.8,23.7,26.100000,2022-06-12T05:10,2022-06-12T20:10,0.0,0.0,0.0,0,22.1,39.2,298.00000,27.02,5.960000,2022-06-12,baku
3456,19/06/2022,1,21.3,11.4,16.200000,17.6,7.4,12.500000,2022-06-19T05:05,2022-06-19T20:45,0.0,0.0,0.0,0,24.2,46.1,116.00000,30.63,6.080000,2022-06-19,villeneuve
3470,03/07/2022,51,18.8,9.1,14.500000,17.5,7.3,12.800000,2022-07-03T04:49,2022-07-03T21:27,0.2,0.2,0.0,2,15.0,29.2,180.00000,20.52,3.510000,2022-07-03,silverstone


In [68]:
print(f1_22_weather.columns.tolist())

['time', 'weathercode (wmo code)', 'temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'temperature_2m_mean (°C)', 'apparent_temperature_max (°C)', 'apparent_temperature_min (°C)', 'apparent_temperature_mean (°C)', 'sunrise (iso8601)', 'sunset (iso8601)', 'precipitation_sum (mm)', 'rain_sum (mm)', 'snowfall_sum (cm)', 'precipitation_hours (h)', 'windspeed_10m_max (km/h)', 'windgusts_10m_max (km/h)', 'winddirection_10m_dominant (°)', 'shortwave_radiation_sum (MJ/m²)', 'et0_fao_evapotranspiration (mm)', 'date', 'circuitRef']


In [70]:
f1_22_weather = f1_22_weather[['date', 'circuitRef', 'time', 'weathercode (wmo code)', 'temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'temperature_2m_mean (°C)', 'apparent_temperature_max (°C)', 'apparent_temperature_min (°C)', 'apparent_temperature_mean (°C)', 'sunrise (iso8601)', 'sunset (iso8601)', 'precipitation_sum (mm)', 'rain_sum (mm)', 'snowfall_sum (cm)', 'precipitation_hours (h)', 'windspeed_10m_max (km/h)', 'windgusts_10m_max (km/h)', 'winddirection_10m_dominant (°)', 'shortwave_radiation_sum (MJ/m²)', 'et0_fao_evapotranspiration (mm)']]

In [71]:
f1_22_weather

,date,circuitRef,time,weathercode (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),temperature_2m_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),apparent_temperature_mean (°C),sunrise (iso8601),sunset (iso8601),precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),precipitation_hours (h),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),shortwave_radiation_sum (MJ/m²),et0_fao_evapotranspiration (mm)
3365,2022-03-20,bahrain,20/03/2022,0,22.8,17.3,20.000000,22.2,14.6,17.900000,2022-03-20T05:41,2022-03-20T17:48,0.0,0.0,0.0,0,22.5,29.2,127.00000,24.07,5.510000
3372,2022-03-27,jeddah,27/03/2022,0,30.4,19.6,24.700000,30.0,19.5,23.600000,2022-03-27T06:21,2022-03-27T18:36,0.0,0.0,0.0,0,28.3,39.2,121.00000,25.53,7.000000
3386,2022-04-10,albert_park,10/04/2022,2,26.4,17.0,21.100000,24.3,15.9,19.700000,2022-04-10T06:42,2022-04-10T18:01,0.0,0.0,0.0,0,25.8,47.2,94.00000,15.47,3.980000
3400,2022-04-24,imola,24/04/2022,61,16.5,10.7,13.700000,14.5,8.8,11.900000,2022-04-24T06:14,2022-04-24T20:08,3.3,3.3,0.0,9,18.0,46.4,246.00000,12.52,2.260000
3414,2022-05-08,miami,08/05/2022,53,30.3,23.3,26.000000,35.7,26.0,29.800000,2022-05-08T06:38,2022-05-08T19:56,2.7,2.7,0.0,11,16.6,34.2,201.00000,21.23,4.390000
3428,2022-05-22,catalunya,22/05/2022,1,30.8,17.1,24.320833,29.8,16.0,23.254166,2022-05-22T06:25,2022-05-22T21:09,0.0,0.0,0.0,0,21.4,35.6,212.98096,29.01,7.179999
3435,2022-05-29,monaco,29/05/2022,53,22.1,18.8,20.800000,24.3,20.0,22.200000,2022-05-29T05:53,2022-05-29T21:02,5.3,5.3,0.0,14,10.3,34.6,321.00000,19.20,3.430000
3449,2022-06-12,baku,12/06/2022,1,27.9,21.4,25.000000,28.8,23.7,26.100000,2022-06-12T05:10,2022-06-12T20:10,0.0,0.0,0.0,0,22.1,39.2,298.00000,27.02,5.960000
3456,2022-06-19,villeneuve,19/06/2022,1,21.3,11.4,16.200000,17.6,7.4,12.500000,2022-06-19T05:05,2022-06-19T20:45,0.0,0.0,0.0,0,24.2,46.1,116.00000,30.63,6.080000
3470,2022-07-03,silverstone,03/07/2022,51,18.8,9.1,14.500000,17.5,7.3,12.800000,2022-07-03T04:49,2022-07-03T21:27,0.2,0.2,0.0,2,15.0,29.2,180.00000,20.52,3.510000
